In [21]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import kernal

# Ej 1

ImportError: cannot import name 'my_function' from 'kernal' (/Users/stig/Desktop/- TU:e/5Q1 ITBA Buenos Aires/72.75 Aprendizarje Automatico/72.75-ML/TP3/kernal.py)